In [1]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


## Holography

In [3]:
import numpy as np
from gpie import Graph, SupportPrior, ConstWave, fft2, AmplitudeMeasurement, mse
from gpie.core.linalg_utils import circular_aperture, masked_random_array
from numpy.typing import NDArray

# ==== Setting Parameter ====
H, W = 128, 128
shape = (H, W)
noise = 1e-4
rng = np.random.default_rng(seed=42)
support_x = circular_aperture(shape=shape, radius=0.2, center=(-0.2, -0.2))
data_x = masked_random_array(support_x, dtype=np.complex128, rng=rng)
support_y =  circular_aperture(shape=shape, radius=0.2, center=(0.2, 0.2))


class Holography(Graph):
    def __init__(self, var: np.float64,  ref_wave: NDArray[np.complex128], support: NDArray[np.bool_]) -> None:
        super().__init__()
        obj = ~SupportPrior(support=support, label="obj", dtype = np.complex64)
        with self.observe():
            meas = AmplitudeMeasurement(var=var) @ (fft2(ref_wave + obj))
        self.compile()


g = Holography(var = noise, ref_wave = data_x, support = support_y)
g.set_init_rng(np.random.default_rng(seed=11))
g.generate_sample(rng=np.random.default_rng(seed=9), update_observed=True)

true_obj = g.get_wave("obj").get_sample()


def monitor(graph, t):
    x = graph.get_wave("obj").compute_belief().data
    err = mse(x, true_obj)
    if t % 10 == 0:
        print(f"[t={t}] PSE (sum) = {err:.5e}")

g.run(n_iter=100, callback=monitor, verbose = False)


[t=0] PSE (sum) = 4.24499e-01
[t=10] PSE (sum) = 1.25432e-03
[t=20] PSE (sum) = 4.09658e-04
[t=30] PSE (sum) = 4.08963e-04
[t=40] PSE (sum) = 4.09031e-04
[t=50] PSE (sum) = 4.09043e-04
[t=60] PSE (sum) = 4.09044e-04
[t=70] PSE (sum) = 4.09045e-04
[t=80] PSE (sum) = 4.09045e-04
[t=90] PSE (sum) = 4.09044e-04


In [5]:
class Holography(Graph):
    def __init__(self, var: np.float64,  ref_wave: NDArray[np.complex128], support: NDArray[np.bool_]) -> None:
        super().__init__()
        obj = ~SupportPrior(support=support, label="obj", dtype = np.complex64)
        with self.observe():
            meas = AmplitudeMeasurement(var=var) @ (fft2(ref_wave + obj))
        self.compile()


g = Holography(var = noise, ref_wave = data_x, support = support_y)

In [11]:
g.get_wave("obj").children[0].output.children[0].output.children[0]

AmplitudeMeas(gen=7, mode=scalar)